### Crime in the time of Corona - Create DB from raw CSV tables

In [1]:
## Add dependencies: Pandas
import pandas as pd
import os # needed to use the os.path.join method to load the files
from sqlalchemy import create_engine # for integrating with PostgreSQL
from config import db_password

#### Incidents: Load raw csv, create dataframe and clean data.

In [ ]:
# Load the CSV Files into a dataframes.
incidents_raw_df = pd.read_csv("../Resources - MPD Data/incidentTableClean.csv")

In [ ]:
# Review the incidents table fields and counts of each field.
incidents_raw_df.info()
# 1788 records. Removed MP20009253 from CSV due to very incomplete record. 

In [ ]:
# Review Number field for uniqueness. Its our primary field
incidents_raw_df.Number.value_counts(sort=True, dropna=True, ascending=False, bins=None)
# Results show 1660 unique values meaning 127 are possible duplicates.
# Looks like Case Number duplication is valid due to multiple offenses under same case.
# Need to create a primary unique field. 

In [ ]:
# Create a primary unique field: Number plus offenseCode. 
# Call it CID for Case Id
incidents_raw_df["CID"] = incidents_raw_df["Number"] + ' - ' + incidents_raw_df["offenseCode"] 
incidents_raw_df.head(3)

In [ ]:
# Check uniqueness of new combined field
incidents_raw_df.CID.value_counts(sort=True, dropna=True, ascending=False, bins=None) 
# Visual review of remaining double entries shows they are in fact duplicates.

In [ ]:
# Drop the Duplicate Case IDs. Then check counts again.
incidents_raw_df.drop_duplicates(subset ="CID", keep = False, inplace = True)
incidents_raw_df.CID.value_counts(sort=True, ascending=False) 

In [ ]:
# Set index to CID.  If it works, we have unique values for our CID field. woohoo!
incidents_raw_df.set_index('CID',inplace=True)
incidents_raw_df.head(3)

In [ ]:
# How much data is left in our dataframe?
incidents_raw_df.info()
# Looks like 1778 rows, so we eliminated 10 records.  Lots of work for a unique primary key, but necessary.

In [ ]:
## Ok. now we review and clean remaining fields in Incidents table. 
#  Start with the date field. Create a new date field stripped to date without time.
incidents_raw_df["dateIncident"] = pd.to_datetime(incidents_raw_df["dateReported"]).apply(lambda x: x.date())
incidents_raw_df.head(3)

In [ ]:
## Create a new week field so we can do stats of crime types over time by week.
incidents_raw_df['weekNumber'] = pd.to_datetime(incidents_raw_df['dateReported']).dt.week

# This works, but since our data starts in 2019, we have five weeks with numbers over 40. Starting with Week 47 with blank week.
incidents_raw_df.weekNumber.value_counts(sort=True, dropna=True, ascending=False, bins=None)

In [ ]:
## How to fix to make weekNumber column useful for linear regression analysis?
#  Add six to each week number so that the first six slots in the week order can be allocated to 2019
incidents_raw_df['weekNumber']= incidents_raw_df['weekNumber'] + 6

In [ ]:
# Now replace the high values for 2019 weeks with proper lower numbers.
incidents_raw_df["weekNumber"].replace({53:1,54:2,55:3,56:4,57:5,58:6}, inplace=True)

In [ ]:
# How does the weekNumber field tally look now? Sweet!
incidents_raw_df.weekNumber.value_counts(sort=True, ascending=False, bins=None)

In [ ]:
## Now we look at our offense codes. 
incidents_raw_df.offenseCode.value_counts(sort=True, ascending=False)
# Need a subject matter expert. This could be ok. Or not.

In [ ]:
# Now export the cleaned Incidents table.

# Crimes to CSV
#cleaned_incidents = os.path.join("CleanData", "Cleaned_Incidents.csv")
cleaned_incidents = "../Resources/CleanedData/Cleaned_Incidents.csv"
incidents_raw_df.to_csv(cleaned_incidents, index=False)

#### FBI Crime Data: Load raw csv, create dataframe and clean data.

In [ ]:
#### FBICrimes Load and Clean the 1980 to 2018 csv data into a dataframe
FBICrimes_df =  pd.read_csv("../Resources/est_crimes_1980_2018_FBI_UCRdata.csv")
FBICrimes_df.head(3)

In [ ]:
## Data looks clean. No obvious transformations needed. Ourarget of aggravated assault has a full row counts of 1974.
FBICrimes_df.info(3)
FBICrimes_df.describe()

In [ ]:
## Exploratory Data Analysis of FBI Crime Data.
#for year in FBICrimes_df:

#### National Unemployment Data,1980- 2018. Load and Clean.

In [ ]:
#### Unemployment Data, 1980-2018. Load and Clean.
Unemployment_df =  pd.read_csv("../Resources/unemployment_by_state_1980_2018_BLSdata.csv")
Unemployment_df.head(3)

In [ ]:
## Data looks clean. No obvious transformations needed. Our target of aggravated assault has a full row counts of 1974.
Unemployment_df.info(3)

#### Suspects: Load raw csv, create dataframe and clean data.

In [ ]:
# Load the suspects csv into a dataframe.
suspects_raw_df = pd.read_csv("../Resources - MPD Data/suspectTable.csv")
suspects_raw_df.head(3)

In [ ]:
# Review the suspects table structure
suspects_raw_df.info()

#### Victims: Load raw csv, create dataframe and clean data.

In [ ]:
# Load the victims csv into a dataframe.
victims_raw_df = pd.read_csv("../Resources - MPD Data/victimTable.csv")

In [ ]:
# Review the suspects table structure
victims_raw_df.info()